Youtube Data v3

In [2]:
from googleapiclient.discovery import build
import re
import pandas as pd
import os

In [3]:
API_KEY = ""

youtube = build('youtube', 'v3', developerKey=API_KEY)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
def clean_newlines(text):
    return " ".join(text.split()).strip()

def remove_emojis(text):
    emoji_pattern = re.compile(
        "["  
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & pictographs
        "\U0001F680-\U0001F6FF"  # Transport & map
        "\U0001F1E0-\U0001F1FF"  # Flags
        "\U00002700-\U000027BF"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

def preprocess_comment(text):
    text = clean_newlines(text)
    text = remove_emojis(text)
    return text.strip()

def get_long_comments(video_id, min_words=25, max_results=100):
    next_page_token = None
    comments = []

    while True:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=max_results,
            pageToken=next_page_token,
            textFormat='plainText'
        ).execute()

        for item in response['items']:
            comment_text = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comment_text = preprocess_comment(comment_text)
            if len(comment_text.split()) >= min_words:
                comments.append(comment_text)

        if 'nextPageToken' in response and len(comments) < max_results:
            next_page_token = response['nextPageToken']
        else:
            break

    return comments

In [ ]:
old_video_ids = ['l54DE-CS1z4', 'Y2s0g9xGWVY', '_cKeLsxwRSU', '4CAQ4odYfhI','z_g5p38fNuU','GoJGVp0m0xk', 'aEWJ6JIQfPE', '6-YEpvc_HJY']

video_ids = ['cNq1xqqBVHQ']
long_comments = []

for vid in video_ids:
    print(f"Mining comment of video: {vid}")
    new_comments = get_long_comments(vid)
    long_comments.extend(new_comments)
    print(f"Mined {len(new_comments)} comment for video {vid}")
    print("First 5 new_comments:", new_comments[:5])



Đang lấy comment dài cho video: cNq1xqqBVHQ
Đã lấy 64 comment dài cho video cNq1xqqBVHQ
Nội dung new_comments: ['Kết truyện thật bất ngờ  đã nào năm qua lầm lỡ lại tiếp tục lỡ lầm Nếu là tôi khi khi ông Quân hỏi “yêu anh thêm lần nữa đc k e?” Chắc t trả lời: “thôi anh à, e ngu 1 lần để phí bao năm thanh xuân, ngu chi thêm lần 2”', '1 người đàn ông tệ bạc như Quân lại được 1 kết cuộc viên mãn hạnh phúc. 1 câu chuyện chẳng có 1 tính nhân văn nào', 'truyện mà làm sao giống thực tế được, nhưng mà nó hợp lý. rút ra bài học: đàn ông tốt kiểu đứng đắng đàng hoàng thì khó có hạnh phúc. dĩ nhiên sẽ có người hợp nhưng rất ít. anh rất tốt nhưng em rất tiếc. phụ nữ không chỉ yêu bằng tấm lòng mà còn yêu bằng tình dục. Quân vừa có tấm lòng vừa có tình dục thì hoàn toàn xứng đáng.', 'Thấy m.n comment mới biết du chua nghe het, ngoại tình lần 1 sẽ có lần 2 vì đó là bản tính. C về với ck cũ dù sống bên nhau nhưng cai gai ngoai tinh vẫn còn đó sớm ngày cũng sẽ đâm chảy máu lần 2', 'khi yêu 1 ng thật lò

In [ ]:
# If the file existed, read the file first
if os.path.exists("data/comments.csv"):
    old_df = pd.read_csv("data/comments.csv", encoding="utf-8-sig")
else:
    old_df = pd.DataFrame(columns=["comment", "emotion"])

# Turn new comment into DataFrame
new_df = pd.DataFrame(long_comments, columns=["comment"])

# Add empty emotion column
if "emotion" not in new_df.columns:
    new_df["emotion"] = ""

new_df = new_df.reindex(columns=old_df.columns, fill_value="")

# Concat
df = pd.concat([old_df, new_df], ignore_index=True)

# Rewirte file
df.to_csv("data/comments.csv", index=False, encoding="utf-8-sig")

UIT-VSMEC

In [3]:
import unicodedata

df_train = pd.read_csv("data/UIT_train.csv")
df_test = pd.read_csv("data/UIT_test.csv")
df_valid = pd.read_csv("data/UIT_valid.csv")

In [4]:
def normalize_vietnamese(text):
    return unicodedata.normalize('NFC', str(text))

def clean_newlines(text):
    return " ".join(text.split()).strip()

def remove_emojis(text):
    """Delete emoji Unicode."""
    emoji_pattern = re.compile(
        "["  
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002700-\U000027BF"  # dingbats
        "\U000024C2-\U0001F251"  # enclosed characters
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

def remove_special_chars(text):
    """Remove special character, link, hashtag, mention."""
    text = re.sub(r"http\S+", "", text)     # link
    text = re.sub(r"www\S+", "", text)      # www
    text = re.sub(r"[@#]\w+", "", text)     # @username, #hashtag
    text = re.sub(r"[^0-9A-Za-zÀ-Ỹà-ỹ\s.,!?]", " ", text)
    return text.strip()

def normalize_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

def preprocess_text(text):
    """Pipeline preprocess text."""
    text = normalize_vietnamese(text)
    text = text.lower()
    text = clean_newlines(text)
    text = remove_emojis(text)
    text = remove_special_chars(text)
    text = normalize_spaces(text)
    return text

In [9]:
def clean_dataset(df):
    df["Clean_sentence"] = df["Sentence"].apply(preprocess_text)
    df = df[df["Clean_sentence"].apply(lambda x: len(x.split()) >= 3)]
    df = df.reset_index(drop=True)
    return df

df_train = clean_dataset(df_train)
df_test = clean_dataset(df_test)
df_valid = clean_dataset(df_valid)

for name, df in [("Train", df_train), ("Test", df_test), ("Valid", df_valid)]:
    print(f"\n📊 {name} set:")
    print(df["Emotion"].value_counts())
    print(f"Sample: {len(df)}")

df_train[["Clean_sentence", "Emotion"]].to_csv("data/UIT_train_clean.csv", index=False, encoding="utf-8-sig")
df_test[["Clean_sentence", "Emotion"]].to_csv("data/UIT_test_clean.csv", index=False, encoding="utf-8-sig")
df_valid[["Clean_sentence", "Emotion"]].to_csv("data/UIT_valid_clean.csv", index=False, encoding="utf-8-sig")

print("\n File is cleaned and saved at: UIT_train_clean.csv")
print("\n File is cleaned and saved at: UIT_test_clean.csv")
print("\n File is cleaned and saved at: UIT_valid_clean.csv")


📊 Train set:
Emotion
Enjoyment    1525
Disgust      1042
Other        1018
Sadness       936
Anger         384
Fear          311
Surprise      239
Name: count, dtype: int64
Sample: 5455

📊 Test set:
Emotion
Enjoyment    185
Other        129
Disgust      128
Sadness      114
Fear          46
Anger         40
Surprise      37
Name: count, dtype: int64
Sample: 679

📊 Valid set:
Emotion
Enjoyment    208
Other        141
Disgust      130
Sadness       85
Anger         48
Fear          31
Surprise      30
Name: count, dtype: int64
Sample: 673

 File is cleaned and saved at: UIT_train_clean.csv

 File is cleaned and saved at: UIT_test_clean.csv

 File is cleaned and saved at: UIT_valid_clean.csv
